In [10]:
import numpy as np
import pandas as pd


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score , cohen_kappa_score, roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from imblearn.combine import SMOTETomek

import warnings
warnings.filterwarnings('ignore')

In [11]:
df = pd.read_pickle('datos/airline_cod_map.pkl')
df.head()

,satisfaccion,genero,tipo_de_cliente,clase,comfort_asiento,comida_y_bebida,ubicación_puerta_embarque,ocio_vuelo,soporte_online,facilidad_reserva_online,...,facturacion,limpieza,embarque_online,retraso_salida_minutos,puntualidad_Insatisfecho,puntualidad_Muy satisfecho,puntualidad_Nada satisfecho,puntualidad_Neutral,puntualidad_Poco Satisfecho,puntualidad_Satisfecho
0,0,1,1,2,1,1,1,1,2,2,...,1,2,2,0,0,0,1,0,0,0
1,0,2,1,1,1,1,3,2,2,2,...,2,2,2,310,0,0,1,0,0,0
2,0,1,1,2,1,1,3,0,2,2,...,1,1,2,0,0,0,1,0,0,0
3,0,1,1,2,1,1,3,1,2,2,...,1,2,1,0,0,0,1,0,0,0
4,0,1,1,2,1,1,3,2,1,2,...,1,2,1,0,0,0,1,0,0,0


<p>Ajustad un modelo de Decision Tree a nuestros datos.</p>

In [12]:
X1 = df.drop("satisfaccion", axis = 1)
y1 = df["satisfaccion"]

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X1, y1, test_size = 0.2, random_state = 42)

In [14]:
param = {"max_depth": [2,4, 6, 10, 12, 14], 
        "max_features": [1,2,3,4],
        "min_samples_split": [10,15,20],
        "min_samples_leaf": [5,10,15]} 

In [15]:
gs_rf = GridSearchCV(
            estimator=RandomForestClassifier(random_state=42),
            param_grid= param,
            cv=10)

In [16]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [2, 4, 6, 10, 12, 14],
                         'max_features': [1, 2, 3, 4],
                         'min_samples_leaf': [5, 10, 15],
                         'min_samples_split': [10, 15, 20]})

In [17]:
bosque = gs_rf.best_estimator_
bosque

RandomForestClassifier(max_depth=14, max_features=4, min_samples_leaf=5,
                       min_samples_split=20, random_state=42)

In [18]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

<p>Calculad las métricas a nuestro nuevo modelo.</p>

In [19]:
def metricas(clases_reales_test, clases_predichas_test, clases_reales_train, clases_predichas_train, modelo):
    
    # para el test
    accuracy_test = accuracy_score(clases_reales_test, clases_predichas_test)
    precision_test = precision_score(clases_reales_test, clases_predichas_test)
    recall_test = recall_score(clases_reales_test, clases_predichas_test)
    f1_test = f1_score(clases_reales_test, clases_predichas_test)
    kappa_test = cohen_kappa_score(clases_reales_test, clases_predichas_test)

    # para el train
    accuracy_train = accuracy_score(clases_reales_train, clases_predichas_train)
    precision_train = precision_score(clases_reales_train, clases_predichas_train)
    recall_train = recall_score(clases_reales_train, clases_predichas_train)
    f1_train = f1_score(clases_reales_train, clases_predichas_train)
    kappa_train = cohen_kappa_score(clases_reales_train, clases_predichas_train)
    

    
    df = pd.DataFrame({"accuracy": [accuracy_test, accuracy_train], 
                       "precision": [precision_test, precision_train],
                       "recall": [recall_test, recall_train], 
                       "f1": [f1_test, f1_train],
                       "kapppa": [kappa_test, kappa_train],
                       "set": ["test", "train"]})
    
    df["modelo"] = modelo
    return df

In [20]:
rf_results = metricas(y_test, y_pred_test_rf,y_train,  y_pred_train_rf, "Random Forest")
rf_results

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.914536,0.930046,0.875717,0.902065,0.826377,test,Random Forest
1,0.920879,0.937763,0.884206,0.910197,0.839596,train,Random Forest


<p>Comparad las métricas con el modelo hecho hasta ahora. ¿Cuál es mejor?</p>

In [22]:
df_LR_results = pd.read_pickle("datos/airline_results_LR.pkl")
df_LR_results

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.872840,0.890668,0.850021,0.869870,0.745679,test,Regresión logistica
1,0.868979,0.883307,0.850290,0.866484,0.737959,train,Regresión logistica


In [23]:
df_DT_results = pd.read_csv('datos/resultados_airline_LR_DT.csv', index_col=0)
df_DT_results

,accuracy,precision,recall,f1,kapppa,set,modelo
0,0.895211,0.897240,0.866039,0.881363,0.787580,test,Decission Tree Esta I
1,0.963813,0.976273,0.943122,0.959411,0.926782,train,Decission Tree Esta I


<p> Hemos analizado los valores de los tres modelos para compararlos. Además hemos establecido precisión como nuestro valor destacado dentro de las métricas ya que estamos priorizando la proporción de predicciones positivas que han sido correctas dentro de nuestro modelo. Dado que a la hora de mapear dimos más peso a la insatisfacción obtenemos como valor positivo de este tipo de clientes. Eso hace que nuestros resultados sigan la lógica de nuestra predicción de modo que debemos comparar los valores de los tres métodos y ver cual es más preciso. 

Esto nos hace concluir que el Random Forest es el método de predicción más preciso por sus valores en precision y kappa.</p>